# 08 - Model Monitoring

This notebook covers configuring model monitoring jobs for skew and dirft detection:
1. Set skew and drift threshold.
2. Create a monitoring job for all the models under and endpoint.
3. List the monitoring jobs.
4. List artifacts produced by monitoring job.
5. Pause and delete the monitoring job.

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2?

In [ ]:
PROJECT = 'ksalama-cloudml' # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us' # Change to your bucket.
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
ENDPOINT_DISPLAY_NAME = 'chicago-taxi-tips-classifier'
MONITORING_JOB_NAME = f"monitor_{ENDPOINT_DISPLAY_NAME}"
NOTIFY_EMAILS = "khalid.m.salama@gmail.com"

LOG_SAMPLE_RATE = 0.8
MONITOR_INTERVAL = 3600
TARGET = 'tip_bin'

In [ ]:
from src.utils.vertex_utils import VertexClient
vertex_client = VertexClient(PROJECT, REGION)

## 1. Set Skew and Drift Thresholds

In [ ]:
SKEW_THRESHOLDS = {
    'trip_month': 0.3,
    'trip_day': 0.3,
    'trip_day_of_week': 0.3,
    'trip_hour': 0.3,
    'trip_seconds': 0.3,
    'trip_miles': 0.3,
    'payment_type': 0.3,
    'pickup_grid': 0.3,
    'dropoff_grid': 0.3,
    'euclidean': 0.3,
    'loc_cross': 0.3,  
}

DIRFT_THRESHOLDS = {
    'trip_month': 0.3,
    'trip_day': 0.3,
    'trip_day_of_week': 0.3,
    'trip_hour': 0.3,
    'trip_seconds': 0.3,
    'trip_miles': 0.3,
    'payment_type': 0.3,
    'pickup_grid': 0.3,
    'dropoff_grid': 0.3,
    'euclidean': 0.3,
    'loc_cross': 0.3,  
}

## 2. Create Monitoring Job

In [ ]:
monitoring_job = vertex_client.create_monitoring_job(
    job_name=MONITORING_JOB_NAME,
    dataset_display_name=DATASET_DISPLAY_NAME,
    endpoint_display_name=ENDPOINT_DISPLAY_NAME,
    target_field=TARGET,
    log_sample_rate=LOG_SAMPLE_RATE,
    monitor_interval=MONITOR_INTERVAL,
    skew_thresholds=SKEW_THRESHOLDS, 
    drift_thresholds=DIRFT_THRESHOLDS,
    notify_emails=NOTIFY_EMAILS
)

monitoring_job

## 3. List Monitoring Jobs

In [ ]:
vertex_client.list_monitoring_jobs()

In [ ]:
monitoring_job = vertex_client.get_monitoring_job_by_name(MONITORING_JOB_NAME)
monitoring_job_artifacts_root = f"gs://{monitoring_job.predict_instance_schema_uri.replace('gs://', '').split('/')[0]}"

In [ ]:
!gsutil ls {monitoring_job_artifacts_root}/*/*

## 5. Pause Monitoring Job

In [ ]:
vertex_client.pause_monitoring_job(MONITORING_JOB_NAME)

## Delete Monitoring Job

In [ ]:
vertex_client.delete_monitoring_job(MONITORING_JOB_NAME)